In [1]:
import sys
sys.path.append("../src/")
from net.modules import ConvEncoder, MultilabelVI, MultilabelBinaryVI, MultilabelMaxI, MultilabelVItrajectory
from utils.utils import *
from utils.data_generator import *
import matplotlib.pyplot as plt
from utils.visual_functions import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
%matplotlib inline

In [2]:
current, voltage, labels, I = get_data( data_type="plaid")

Load plaid aggregated data from ../data/plaid/


In [3]:
I_max  = compute_active_non_active_features(current, voltage, emb_size=50)  
input_feature = generate_input_feature(current, voltage, 'vi', 50, True) 

frze processed: 100 percent: : 1154it [00:50, 22.80it/s]
processed: 1154: : 1154it [00:05, 212.98it/s]


In [7]:
img_tra=np.concatenate([input_feature.reshape(len(input_feature), -1), I_max.reshape(len(I_max), -1)], 1)
img_tra.shape                    

(1154, 2600)

In [ ]:
def fryze_power_decomposition(i, v, T=500):
    pact = i[:,None]*v[:, None]
    pact = np.sum(pact)/T
    vrms = np.sum(v[:,None]**2)/T
    i_active = pact*v/vrms
    i_non_active = i- i_active 
    return i_active, i_non_active


def compute_active_non_active_features(current, voltage, emb_size=50):
    n = len(current)
    features = np.zeros((n, emb_size, 2))
    with tqdm(n) as pbar:
        for k in range(n):
            i_active, i_non_active = fryze_power_decomposition(current[k], voltage[k], T=500)
            i_active=paa(i_active.flatten(), emb_size)
            i_non_active=paa(i_non_active.flatten(), emb_size)
            features[k]=np.hstack([i_non_active[:,None], i_active[:,None]]).astype(float)
            pbar.set_description('frze processed: %d percent' % round((1 + k)*100/n, 2))
            pbar.update(1)
        pbar.close() 
    features = torch.tensor(features).float()
    features = features.view(-1, 2, emb_size)
    return features

In [ ]:
features = compute_active_non_active_features(current, voltage, emb_size=50)

In [ ]:
plt.subplot(3,1,1)
plt.plot(i, v)
plt.subplot(3,1,2)
plt.plot(ia, v)
plt.subplot(3,1,3)
plt.plot(ip, v)

In [ ]:

emb_size=50

inputs = []
for i in range(len(current)):
    series = current[i].T
    dist = torch.empty(d, emb_size)
    d, T =  series.shape
    dist = []
    for k in range(d):
         dist+=[create_paa(series[k,:], emb_size)]
    inputs+=[dist]

 

In [ ]:
inputs

In [ ]:
paa_curent=create_paa(current, 50)

In [ ]:
paa_curent.shape

In [ ]:
model=MultilabelVItrajectory(in_size=1, out_size=12)

In [ ]:
x=paa_curent[:4].unsqueeze(1)
x.shape

In [ ]:
model(x, None)

In [ ]:
input_feature = generate_input_feature(current, voltage, "vi", 50, True)

In [ ]:
np.unique(np.hstack(labels))